In [2]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('bus_trafic_clean.csv')
df.columns

Index(['Unnamed: 0', 'horodatage', 'identifiant_SAE_de_desserte',
       'identifiant_du_vehicule', 'numero_Timeo_de_l_arret',
       'etat_SAE_du_vehicule', 'numero_de_parc_du_vehicule', 'nom_de_la_ligne',
       'Heure_estimee_de_passage_a_L_arret', 'modele_du_vehicule',
       'identifiant_SAE_de_ligne', 'ecart_horaire_en_secondes', 'destination',
       'nom_de_l_arret', 'mne_de_l_arret', 'mnemo_de_la_ligne',
       'coordonnees_GPS_WG84', 'identifiant_SAE_de_l_arret',
       'cap_du_vehicule_en_degres', 'identifiant_SAE_du_parcours',
       'service_voiture', 'coordonnees_GPS_X', 'coordonnees_GPS_Y',
       'horodatage_maj', 'cordonnees_bus_geometrie', 'ecart_horodatage',
       'latitude', 'longitude', 'year', 'month', 'day', 'hours', 'minutes',
       'date', 'date_heure', 'jour_semaine', 'horodatage_round',
       'MAX_TEMPERATURE_C', 'MIN_TEMPERATURE_C', 'WINDSPEED_MAX_KMH',
       'TEMPERATURE_MORNING_C', 'TEMPERATURE_NOON_C', 'TEMPERATURE_EVENING_C',
       'PRECIP_TOTAL_DAY

In [56]:
df_pred = df[['ecart_horaire_en_secondes', 
            'etat_SAE_du_vehicule', 
            #'nom_de_la_ligne',
            #'mne_de_l_arret',
            #'latitude', 'longitude', 
            #'Heure_estimee_de_passage_a_L_arret',
            #'year', 
            'month', 
            'day', 
            'hours', 
            'minutes', 
            #'jour_semaine', 
            #'OPINION'
            ]]

#df_pred = df_pred.astype({"Heure_estimee_de_passage_a_L_arret": "datetime64"})
#df_pred = df_pred.astype({"identifiant_SAE_de_ligne": "object"})       

#df_pred["year_passage_a_l_arret"] = df_pred["Heure_estimee_de_passage_a_L_arret"].dt.year
#df_pred["month_passage_a_l_arret"] = df_pred["Heure_estimee_de_passage_a_L_arret"].dt.month
#df_pred["day_passage_a_l_arret"] = df_pred["Heure_estimee_de_passage_a_L_arret"].dt.day
#df_pred["hours_passage_a_l_arret"] = df_pred["Heure_estimee_de_passage_a_L_arret"].dt.hour
#df_pred["minutes_passage_a_l_arret"] = df_pred["Heure_estimee_de_passage_a_L_arret"].dt.minute
#df_pred.drop(columns=['Heure_estimee_de_passage_a_L_arret'], inplace=True)

df_pred.shape


(724202, 6)

In [57]:
df_pred = pd.get_dummies(df_pred)
df_pred.shape

(724202, 18)

In [58]:
# corr df_pred
corr = df_pred.corr()
corr.style.background_gradient(cmap='coolwarm')

,ecart_horaire_en_secondes,month,day,hours,minutes,etat_SAE_du_vehicule_DEV,etat_SAE_du_vehicule_DEVP,etat_SAE_du_vehicule_GARE,etat_SAE_du_vehicule_HC,etat_SAE_du_vehicule_HL,etat_SAE_du_vehicule_HLP,etat_SAE_du_vehicule_HLPR,etat_SAE_du_vehicule_HLPS,etat_SAE_du_vehicule_HS,etat_SAE_du_vehicule_INC,etat_SAE_du_vehicule_LIGN,etat_SAE_du_vehicule_TARR,etat_SAE_du_vehicule_TDEP
ecart_horaire_en_secondes,1.000000,0.077272,-0.022302,0.068165,0.027198,0.022371,0.023143,-0.001482,-0.009675,0.035356,-0.008573,-0.075138,0.014263,0.001656,-0.013475,0.175796,-0.126758,-0.167589
month,0.077272,1.000000,-0.237488,0.075827,-0.004185,0.033624,0.019009,0.002144,0.005366,0.011611,0.002097,0.007895,0.024937,0.000162,-0.003775,-0.006203,-0.018100,-0.019379
day,-0.022302,-0.237488,1.000000,-0.000892,0.001446,0.006695,0.013404,-0.001061,-0.003484,-0.007331,-0.001391,-0.003303,-0.009202,0.001350,-0.003290,-0.006894,0.006662,0.005225
hours,0.068165,0.075827,-0.000892,1.000000,-0.066230,0.007863,0.005408,0.000968,-0.003296,0.004297,-0.000369,0.097764,-0.083782,-0.001302,0.000466,0.024051,-0.013709,-0.026780
minutes,0.027198,-0.004185,0.001446,-0.066230,1.000000,0.001293,0.011386,-0.000002,-0.004622,0.006782,-0.001336,-0.009268,-0.014350,-0.003358,0.005346,0.053655,-0.040425,-0.045992
etat_SAE_du_vehicule_DEV,0.022371,0.033624,0.006695,0.007863,0.001293,1.000000,-0.013109,-0.000181,-0.003127,-0.006539,-0.005159,-0.009171,-0.012117,-0.000128,-0.002281,-0.147042,-0.017486,-0.022655
etat_SAE_du_vehicule_DEVP,0.023143,0.019009,0.013404,0.005408,0.011386,-0.013109,1.000000,-0.000400,-0.006917,-0.014463,-0.011411,-0.020284,-0.026799,-0.000283,-0.005044,-0.325223,-0.038675,-0.050107
etat_SAE_du_vehicule_GARE,-0.001482,0.002144,-0.001061,0.000968,-0.000002,-0.000181,-0.000400,1.000000,-0.000095,-0.000200,-0.000158,-0.000280,-0.000370,-0.000004,-0.000070,-0.004489,-0.000534,-0.000692
etat_SAE_du_vehicule_HC,-0.009675,0.005366,-0.003484,-0.003296,-0.004622,-0.003127,-0.006917,-0.000095,1.000000,-0.003450,-0.002722,-0.004839,-0.006393,-0.000068,-0.001203,-0.077584,-0.009226,-0.011953
etat_SAE_du_vehicule_HL,0.035356,0.011611,-0.007331,0.004297,0.006782,-0.006539,-0.014463,-0.000200,-0.003450,1.000000,-0.005692,-0.010118,-0.013368,-0.000141,-0.002516,-0.162224,-0.019291,-0.024994


In [52]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = df_pred.drop(columns=['ecart_horaire_en_secondes'])
#X = scaler.fit_transform(X)

y = df_pred['ecart_horaire_en_secondes']
y = df_pred['ecart_horaire_en_secondes'].apply(lambda x: 1 if x > 0 else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [53]:
# arbre 
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.7267348333690047
Confusion Matrix:
[[29789 21614]
 [17966 75472]]


In [54]:
y = df_pred['ecart_horaire_en_secondes']

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np


lr= LinearRegression()

lr.fit(X_train, y_train)
ypred = lr.predict(X_test)

r2_score(y_test, ypred), mean_absolute_error(y_test, ypred), np.sqrt(mean_squared_error(y_test, ypred)), mean_absolute_percentage_error(y_test, ypred)


(0.27539875723816953,
 0.33309784217863864,
 0.4072996367708134,
 753222603623182.5)

In [43]:
# decision tree regressor
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [44]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

print("MSE: ", mean_squared_error(y_test, y_pred))
print("MAE: ", mean_absolute_error(y_test, y_pred))
print("MAPE: ", mean_absolute_percentage_error(y_test, y_pred))

MSE:  0.2064391346525696
MAE:  0.29455894267851485
MAPE:  662728497332611.4


In [ ]:
# decision tree regressor
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [ ]:
print("MSE: ", mean_squared_error(y_test, y_pred))
print("MAE: ", mean_absolute_error(y_test, y_pred))
print("MAPE: ", mean_absolute_percentage_error(y_test, y_pred))